In [1]:
import os
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score

# config paths
# attention chemin corrige suite au debug precedent
PROJECT_ROOT = r"C:\Users\amisf\Desktop\datascientest_projet"
IMG_DIR = os.path.join(PROJECT_ROOT, "data", "raw", "images", "images", "image_train")
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "implementation", "outputs")

# params light pr tourner partout
BATCH_SIZE = 32 
IMG_SIZE = (224, 224) # standard efficientnet b0
EPOCHS = 5
LR = 1e-3

# auto-switch gpu/cpu
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"🚀 mode gpu activé : {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("⚠️ pas de gpu détecté. passage en mode cpu (sera plus lent mais fonctionnel)")

🚀 mode gpu activé : NVIDIA GeForce RTX 4070


In [2]:
# load csvs
csv_path = os.path.join(PROJECT_ROOT, "data", "raw")
df_x = pd.read_csv(os.path.join(csv_path, "X_train_update.csv"), index_col=0)
df_y = pd.read_csv(os.path.join(csv_path, "Y_train_CVw08PX.csv"), index_col=0)
df = pd.merge(df_x, df_y, left_index=True, right_index=True)

# chemin img
df['filename'] = df.apply(lambda x: f"image_{x['imageid']}_product_{x['productid']}.jpg", axis=1)
df['path'] = df['filename'].apply(lambda x: os.path.join(IMG_DIR, x))

# encode labels
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['prdtypecode'])
num_classes = len(le.classes_)

# split train/val
# on garde un bon morceau pr valider le score
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label_encoded'], random_state=42)

print(f"✅ data ready : {len(train_df)} train / {len(val_df)} val")

# dataset class
class SimpleDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        try:
            # conversion rgb obligatoire
            img = Image.open(row['path']).convert("RGB")
        except:
            # fallback noir si erreur fichier
            img = Image.new('RGB', IMG_SIZE, (0, 0, 0))
            
        if self.transform:
            img = self.transform(img)
            
        return img, torch.tensor(row['label_encoded'], dtype=torch.long)

# transformations standard (pas trop lourd pr cpu)
trans = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# loaders
# num_workers=0 pr eviter deadlock windows si cpu
train_ds = SimpleDataset(train_df, transform=trans)
val_ds = SimpleDataset(val_df, transform=trans)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

✅ data ready : 67932 train / 16984 val


In [3]:
print("🛠️ init modele classique (efficientnet_b0)...")

# load poids imagenet par defaut
model = models.efficientnet_b0(weights="DEFAULT")

# freeze backbone (gain temps enorme + moins lourd pr cpu)
for param in model.features.parameters():
    param.requires_grad = False
    
# replace head pr nos 27 classes
in_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_features, num_classes)

model = model.to(device)
print(f"✅ modele chargé sur {device}")

🛠️ init modele classique (efficientnet_b0)...
✅ modele chargé sur cuda


In [5]:
# --- OPTIMISATION : FINE TUNING (DEBRIDAGE) ---
# on debloque tout le modele pour qu'il apprenne vraiment
print("🔓 déblocage du modele complet pour booster le score...")

for param in model.parameters():
    param.requires_grad = True

# config "douce" pour ne pas casser les poids existants
# on passe en AdamW (meilleur) avec un learning rate faible
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

# scheduler : si ca stagne 2 epochs, on divise le lr par 5
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='max', factor=0.2, patience=2
)

criterion = nn.CrossEntropyLoss()
history = {'f1': []}
best_f1 = 0.0

print("🔥 start fine-tuning (10 epochs max)...")
print("🎯 objectif : > 0.75")

for epoch in range(10):
    model.train()
    loss_ep = 0.0
    t0 = time.time()
    
    for i, (imgs, lbls) in enumerate(train_loader):
        imgs, lbls = imgs.to(device), lbls.to(device)
        
        optimizer.zero_grad()
        # autocast pas obligatoire ici car modele leger, mais bon si gpu dispo
        if device.type == 'cuda':
            with torch.cuda.amp.autocast():
                out = model(imgs)
                loss = criterion(out, lbls)
            
            # backward classique (sans scaler si pas amp, ou avec scaler)
            # ici on fait simple sans scaler complexe vu que c'est le "classique"
            loss.backward()
        else:
            out = model(imgs)
            loss = criterion(out, lbls)
            loss.backward()
            
        optimizer.step()
        loss_ep += loss.item()
        
        if i % 100 == 0:
            print(f"   ep {epoch+1} | batch {i} | loss {loss.item():.4f}", end="\r")
            
    # validation
    model.eval()
    preds, targets = [], []
    with torch.no_grad():
        for imgs, lbls in val_loader:
            imgs = imgs.to(device)
            out = model(imgs)
            _, p = torch.max(out, 1)
            preds.extend(p.cpu().numpy())
            targets.extend(lbls.cpu().numpy())
            
    val_f1 = f1_score(targets, preds, average='weighted')
    duree = time.time() - t0
    avg_loss = loss_ep / len(train_loader)
    
    # update scheduler
    scheduler.step(val_f1)
    current_lr = optimizer.param_groups[0]['lr']
    
    print(f"\n✅ end ep {epoch+1} | time {duree:.0f}s | loss {avg_loss:.4f} | f1 {val_f1:.4f} | lr {current_lr:.6f}")
    
    # save best
    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.state_dict(), os.path.join(OUTPUT_DIR, "classic_efficientnet_b0.pth"))
        print("   💾 new best score saved.")
        
    # stop si score suffisant (on veut juste un modele decent)
    if val_f1 > 0.82:
        print("🚀 score cible atteint. on arrete.")
        break

print(f"🏁 fin training classique. meilleur f1 : {best_f1:.4f}")

🔓 déblocage du modele complet pour booster le score...
🔥 start fine-tuning (10 epochs max)...
🎯 objectif : > 0.75


C:\Users\amisf\AppData\Local\Temp\ipykernel_49576\1046154534.py:35: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


   ep 1 | batch 2100 | loss 1.1315
✅ end ep 1 | time 485s | loss 1.4413 | f1 0.6278 | lr 0.000100
   💾 new best score saved.


C:\Users\amisf\AppData\Local\Temp\ipykernel_49576\1046154534.py:35: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


   ep 2 | batch 2100 | loss 0.8902
✅ end ep 2 | time 535s | loss 1.1139 | f1 0.6536 | lr 0.000100
   💾 new best score saved.


...

   ep 8 | batch 2100 | loss 0.0851
✅ end ep 8 | time 481s | loss 0.3620 | f1 0.6692 | lr 0.000100


C:\Users\amisf\AppData\Local\Temp\ipykernel_49576\1046154534.py:35: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


   ep 9 | batch 2100 | loss 0.2423
✅ end ep 9 | time 483s | loss 0.3001 | f1 0.6689 | lr 0.000020


C:\Users\amisf\AppData\Local\Temp\ipykernel_49576\1046154534.py:35: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


   ep 10 | batch 2100 | loss 0.3172
✅ end ep 10 | time 483s | loss 0.2046 | f1 0.6788 | lr 0.000020
   💾 new best score saved.
🏁 fin training classique. meilleur f1 : 0.6788


In [6]:
import json

#  EXPORT LIVRABLE CLASSIQUE 
print("\n📦 packaging modele classique...")

out_dir = os.path.join(PROJECT_ROOT, "implementation", "outputs")
final_path = os.path.join(out_dir, "livrable_model_classique_effnetb0.pth")

# sauvegarde poids
torch.save(model.state_dict(), final_path)

# sauvegarde meta
classes_mapping = {int(i): str(c) for i, c in enumerate(le.classes_)}
meta_data = {
    "model_name": "EfficientNet-B0 Classique",
    "input_size": [224, 224],
    "num_classes": num_classes, # attention variable minuscule dans ce notebook
    "class_mapping": classes_mapping,
    "description": "modele leger pour cpu/mobile"
}

with open(os.path.join(out_dir, "livrable_classique_metadata.json"), 'w') as f:
    json.dump(meta_data, f, indent=4)

print(f"✅ livrable classique prêt : {final_path}")


📦 packaging modele classique...
✅ livrable classique prêt : C:\Users\amisf\Desktop\datascientest_projet\implementation\outputs\livrable_model_classique_effnetb0.pth
